In [1]:
# From https://www.youtube.com/watch?v=ztrkbh4mNaA

#!pip3 install carla
import carla
import random
import time

# Connect to the client and retrieve the world object
client = carla.Client('localhost', 2000)

world = client.get_world()

"""
# Set up the simulator in synchronous mode
settings = world.get_settings()
settings.synchronous_mode = True # Enables synchronous mode
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)
world.tick()

# Set up the TM in synchronous mode
traffic_manager = client.get_trafficmanager()
traffic_manager.set_synchronous_mode(True)

# Set a seed so behaviour can be repeated if necessary
traffic_manager.set_random_device_seed(0)
random.seed(0)
#"""

# Retrieve the spectator object
spectator = world.get_spectator()

spawn_points = world.get_map().get_spawn_points()

bp_lib = world.get_blueprint_library()


In [2]:
def run_iteration(runtime=30.):
    
    vehicle_bp = bp_lib.find("vehicle.lincoln.mkz_2020")
    vehicle = world.spawn_actor(vehicle_bp, random.choice(spawn_points))

    # Move to the spectator place
    ego_transf = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)), vehicle.get_transform().rotation)
    spectator.set_transform(ego_transf)

    # Create a transform to place the camera on top of the vehicle
    camera_init_trans = carla.Transform(carla.Location(z=1.6, x=0.4))
    # We create the camera through a blueprint that defines its properties
    camera_rgb_bp = bp_lib.find('sensor.camera.rgb')
    # We spawn the camera and attach it to our ego vehicle
    camera_rgb = world.spawn_actor(camera_rgb_bp, camera_init_trans, attach_to=vehicle)
    # We create the camera through a blueprint that defines its properties
    camera_ss_bp = bp_lib.find('sensor.camera.semantic_segmentation')
    # We spawn the camera and attach it to our ego vehicle
    camera_ss = world.spawn_actor(camera_ss_bp, camera_init_trans, attach_to=vehicle)

    # Spawn vehicles
    vehicle_blueprints = bp_lib.filter('*vehicle*')

    # Spawn 50 vehicles randomly distributed throughout the map 
    # for each spawn point, we choose a random vehicle from the blueprint library
    for i in range(0,50):
        world.try_spawn_actor(random.choice(vehicle_blueprints), random.choice(spawn_points))

    camera_rgb.listen(lambda image: image.save_to_disk('CARLA_data/Images/%06d.png' % image.frame))
    camera_ss.listen(lambda image: image.save_to_disk('CARLA_data/Masks/%06d.png' % image.frame)) 

    # Set Traffic Manager autopilot for all cars
    for vehicle in world.get_actors().filter('vehicle*'):
        vehicle.set_autopilot(True)
    
    """
    while True:
        world.tick()
    """

    time.sleep(runtime)
    camera_rgb.stop()
    camera_ss.stop()

    for vehicle in world.get_actors().filter('vehicle*'):
        vehicle.destroy()
    for cam in world.get_actors().filter('camera*'):
        cam.destroy()

In [3]:
for i in range(5):
    run_iteration()


KeyboardInterrupt



In [ ]:
# Always disable sync mode before the script ends to prevent the server blocking whilst waiting for a tick
settings.synchronous_mode = False
world.apply_settings(settings)
traffic_manager.set_synchronous_mode(False)